### This code creates the results used in Section 6.3

In [ ]:
### Some initializations
import numpy as np
import random
import loli
import pickle
from scipy.stats import binomtest


In [ ]:
# The function that generates the dynamical data

def lordata_gen(size):
    X=np.zeros((size+1,6))
    X[0,:]=[2,0.97,0.99,1,0.97,1]
    
    
    for t in range(size):
        e=np.random.multivariate_normal(np.zeros(6),np.eye(6))
        x_new=[X[t,0]*0.9+0.1*X[t,1]+e[0],\
               0.28*X[t,0]-0.01*X[t,0]*X[t,2]+0.99*X[t,1]+e[1],\
               0.01*X[t,0]*X[t,1]-0.01*X[t,0]*X[t,3]+0.9733*X[t,2]+e[2],\
               0.01*X[t,0]*X[t,2]-0.02*X[t,0]*X[t,4]+0.9366*X[t,3]+e[3],\
               0.02*X[t,0]*X[t,3]+0.96*X[t,4]+e[4],
                  X[t,5]+e[5]]
        X[t+1,:]=x_new

    return(X)


In [ ]:
# Running our algorithm on the Lorenz data

rs=1
np.random.seed(rs)
size=8500 # The size of the generated dataset
E=300 # The number of environments
runs=500 # How often we repeat the experiment 
results={}
for n in [20,25]:  # n is the sample size (length of the interval)
    for tar in range(6):
        results[tar]={}
    for j in range(runs):
        print('Run',j,'out of',runs)
        X=lordata_gen(size)
        dataX=[X[499+i*n:499+(i+1)*n,:] for i in range(E)]
        for tar in range(6):
            dataY=[X[499+i*n+1:499+(i+1)*n+1,tar] for i in range(E)]
            plausibleS=loli.gauss(dataX,dataY,alpha=0.1,B=500)
            if len(plausibleS)==0:
                supphat=set()
            else:
                supphat=set.intersection(*plausibleS)
            if tuple(supphat) in results[tar]:
                results[tar][tuple(supphat)]+=1
            else:
                results[tar][tuple(supphat)]=1

    with open('LorenzGenResultsE='+str(E)+'N='+str(n)+'.pkl','wb') as f:
        pickle.dump(results, f)

Run 453 out of 500
Run 454 out of 500
Run 455 out of 500
Run 456 out of 500
Run 457 out of 500
Run 458 out of 500
Run 459 out of 500
Run 460 out of 500
Run 461 out of 500
Run 462 out of 500
Run 463 out of 500
Run 464 out of 500
Run 465 out of 500
Run 466 out of 500
Run 467 out of 500
Run 468 out of 500
Run 469 out of 500
Run 470 out of 500
Run 471 out of 500
Run 472 out of 500
Run 473 out of 500
Run 474 out of 500
Run 475 out of 500
Run 476 out of 500
Run 477 out of 500
Run 478 out of 500
Run 479 out of 500
Run 480 out of 500
Run 481 out of 500
Run 482 out of 500
Run 483 out of 500
Run 484 out of 500
Run 485 out of 500
Run 486 out of 500
Run 487 out of 500
Run 488 out of 500
Run 489 out of 500
Run 490 out of 500
Run 491 out of 500
Run 492 out of 500
Run 493 out of 500
Run 494 out of 500
Run 495 out of 500
Run 496 out of 500
Run 497 out of 500
Run 498 out of 500
Run 499 out of 500
Run 0 out of 500
Run 1 out of 500
Run 2 out of 500
Run 3 out of 500
Run 4 out of 500
Run 5 out of 500
Run 6

Run 390 out of 500
Run 391 out of 500
Run 392 out of 500
Run 393 out of 500
Run 394 out of 500
Run 395 out of 500
Run 396 out of 500
Run 397 out of 500
Run 398 out of 500
Run 399 out of 500
Run 400 out of 500
Run 401 out of 500
Run 402 out of 500
Run 403 out of 500
Run 404 out of 500
Run 405 out of 500
Run 406 out of 500
Run 407 out of 500
Run 408 out of 500
Run 409 out of 500
Run 410 out of 500
Run 411 out of 500
Run 412 out of 500
Run 413 out of 500
Run 414 out of 500
Run 415 out of 500
Run 416 out of 500
Run 417 out of 500
Run 418 out of 500
Run 419 out of 500
Run 420 out of 500
Run 421 out of 500
Run 422 out of 500
Run 423 out of 500
Run 424 out of 500
Run 425 out of 500
Run 426 out of 500
Run 427 out of 500
Run 428 out of 500
Run 429 out of 500
Run 430 out of 500
Run 431 out of 500
Run 432 out of 500
Run 433 out of 500
Run 434 out of 500
Run 435 out of 500
Run 436 out of 500
Run 437 out of 500
Run 438 out of 500
Run 439 out of 500
Run 440 out of 500
Run 441 out of 500
Run 442 out 

ValueError: df <= 0

In [ ]:
# Showing the results, different rows correspond to different targets and different columns to the potential causal parents

E=300
for n in [20,25]:
    with open('LorenzGenResultsE='+str(E)+'N='+str(n)+'.pkl','rb') as f:
        Lorenz=pickle.load(f)
    
    
    final=np.zeros((6,6))
    for tar in range(6):
        for key in Lorenz[tar]:
            for var in range(6):
                if var in key:
                    final[tar,var]+=Lorenz[tar][key]
    print('Raw counts for n=',n)
    print(final)
    pvals=np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            test=binomtest(np.int32(final[i,j]),500,0.1,alternative='greater')
            pvals[i,j]=test.pvalue
    print('Maxtrix of causal parents for n=',n)
    print(pvals<0.05)

In [ ]:
# Alternatively we can directly load the results from the saved file

E=300
for n in [20,25]:
    with open('SavedResults/LorenzData/LorenzGenResultsE='+str(E)+'N='+str(n)+'.pkl','rb') as f:
        Lorenz=pickle.load(f)
    
    
    final=np.zeros((6,6))
    for tar in range(6):
        for key in Lorenz[tar]:
            for var in range(6):
                if var in key:
                    final[tar,var]+=Lorenz[tar][key]
    print('Raw counts for n=',n)
    print(final)
    pvals=np.zeros((6,6))
    for i in range(6):
        for j in range(6):
            test=binomtest(np.int32(final[i,j]),500,0.1,alternative='greater')
            pvals[i,j]=test.pvalue
    print('Maxtrix of causal parents for n=',n)
    print(pvals<0.05)